## Trabajo No. 3

### Sobre: Lógica borrosa
#### Integrantes: Cristian Yepes Sanchez, Edison Escobar Posada

Para la solución del problema planteado, encontramos un Dataset público (https://www.kaggle.com/c/instacart-market-basket-analysis), este Dataset nos proporciona una amplia información sobre las compras de los productos para el hogar, productos para las mascotas entre otros productos. Con este Dataset se pretende extraer información que me permita deducir, si dos usuarios elegidos aleatoriamente, son similares o no, dependiendo de sus preferencias históricas de compras realizadas, en este caso en la plataforma de www.instacart.com

#### Explicación Dataset
El Dataset que elegimos está compuesto por la siguiente información:
### 1. ailes.
Son los pasillos del supermercado en el cual se encuentra el producto, dentro de este encontramos un identificador único y el nombre del pasillo.
### 2. departments.
Este archivo son las clasificaciones de los productos, por ejemplo beberages, alcohol entre otros. En este podemos encontrar su identificador único y el nombre clasificatorio.
### 3. products.
Acá encontramos la información de los productos y esta se encuentra compuesta por el identificador único, nombre, en que pasillo se encuentra y como está clasificado el producto.